In [1]:
import findspark
#useful to install this tool to simplify spark import
findspark.init()
from pyspark import  SparkContext
sc = SparkContext( 'local', 'pyspark')
import numpy

In [17]:
from math import sqrt

def sim_pearson(ratings1,ratings2):
    ratings_common=[]
    for rate1 in ratings1:
        for rate2 in ratings2:
            if (rate1[0]==rate2[0]):
                ratings_common.append([rate1[1],rate2[1]])

    n=len(ratings_common)
    if (n==0):
        return 0
    
    sum1=float(sum([rate[0] for rate in ratings_common]))
    sum2=float(sum([rate[1] for rate in ratings_common]))
    
    sum1sq=float(sum([pow(rate[0],2) for rate in ratings_common]))
    sum2sq=float(sum([pow(rate[1],2) for rate in ratings_common]))
    
    pSum=float(sum([rate[0]*rate[1] for rate in ratings_common]))
    
    num=pSum-(sum1*sum2/n)
    den=sqrt((sum1sq-pow(sum1,2)/n)*(sum2sq-pow(sum2,2)/n))
    if den==0: return 0
    
    return num/den

    

In [18]:
rdd = sc.textFile("train_no_header.csv")
rdd = rdd.map( lambda x: x.split(',') )
rdd = rdd.map(lambda x:(x[0],[x[1],int(x[2])]))

rdd = rdd.groupByKey()
rdd = rdd.map(lambda x:(x[0],list(x[1])))
sim_rdd=rdd.cartesian(rdd)
sim_rdd=sim_rdd.filter(lambda x: x[0][0] > x[1][0])
sim_rdd=sim_rdd.map(lambda x: (x[0][0],x[1][0],sim_pearson(x[0][1],x[1][1]))).filter(lambda x:x[2]>0)

In [20]:
sim_rdd.take(1000)

[(u'11543', u'10109', 1.0),
 (u'11540', u'10654', 1.0),
 (u'11549', u'11465', 1.0),
 (u'3383', u'12725', 1.0),
 (u'5988', u'12015', 0.7573172464461015),
 (u'6342', u'12015', 0.6673083711820307),
 (u'5988', u'3518', 1.0),
 (u'6342', u'2682', 0.5000000000000213),
 (u'5989', u'15280', 0.5000000000000024),
 (u'5988', u'3748', 0.5755412232575111),
 (u'5988', u'4668', 0.8017837257372732),
 (u'5988', u'365', 1.0),
 (u'5988', u'5869', 0.8528028654224417),
 (u'5988', u'5862', 1.0),
 (u'5988', u'12209', 1.0),
 (u'5988', u'13868', 0.2544566789039913),
 (u'6342', u'152', 1.0),
 (u'6342', u'11138', 0.674199862463242),
 (u'5988', u'5070', 0.5683820811809945),
 (u'5988', u'15322', 1.0),
 (u'6342', u'1726', 0.8660254037844402),
 (u'5988', u'219', 1.0),
 (u'5988', u'4780', 1.0),
 (u'5988', u'10109', 1.0),
 (u'6342', u'11182', 1.0),
 (u'5988', u'11581', 1.0),
 (u'5988', u'11872', 1.0),
 (u'5988', u'14342', 0.4677071733467415),
 (u'5988', u'2823', 0.6674238124719146),
 (u'5988', u'12752', 0.2979270115341